<a href="https://colab.research.google.com/github/diego-andrade279/Aula-de-deploy-de-modelos-de-machine-learning-usando-streamlit/blob/main/Regress%C3%A3o_Aula_II.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from pydrive.auth import GoogleAuth 
from pydrive.drive import GoogleDrive 
from google.colab import auth 
from oauth2client.client import GoogleCredentials 
import statsmodels.api as sm
import xlrd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_squared_log_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import (RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor)
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.tree import export_graphviz
import pydot
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from pandas_profiling import ProfileReport


import warnings
warnings.filterwarnings('ignore')

In [ ]:
auth.authenticate_user() 
gauth = GoogleAuth() 
gauth.credentials = GoogleCredentials.get_application_default() 
drive = GoogleDrive(gauth)

In [ ]:
# Baixa os dados de treinamento
link = 'https://drive.google.com/file/d/1qdnOt7JuOZE3_O13T2CROPS12QFa15WT/view?usp=sharing'
id = link.split("/")[-2]
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('house prices train.csv')
train = pd.read_csv('house prices train.csv')
train.shape

(1460, 81)

In [ ]:
# Baixa os dados de teste
link = 'https://drive.google.com/file/d/1gv3uFRhTRdCfDktOy2QR1xnlOquKFzib/view?usp=sharing'
id = link.split("/")[-2]
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('house prices test.csv')

test = pd.read_csv('house prices test.csv')
test.shape

(1459, 80)

In [ ]:
# cria variável SalePrice no dataset de test para facilitar manipulação dos dados de treino e teste juntos
test['SalePrice'] = 0

In [ ]:
# Junta os dois datasets
frames = [train, test]
df = pd.concat(frames)

In [ ]:
percent_missing = df.isnull().sum() * 100 / len(df) # Soma dos dados faltantes divido pela quantidade de registros para calcular percentual
# gera um dataset com duas colunas colunm name e missing percent
missing_value_df = pd.DataFrame({'column_name': df.columns, 
                                 'percent_missing': percent_missing})
missing_value_df.sort_values(['percent_missing'], ascending=False) # apresenta ordenado do maior para o menor 

,column_name,percent_missing
PoolQC,PoolQC,99.657417
MiscFeature,MiscFeature,96.402878
Alley,Alley,93.216855
Fence,Fence,80.438506
FireplaceQu,FireplaceQu,48.646797
LotFrontage,LotFrontage,16.649538
GarageFinish,GarageFinish,5.447071
GarageQual,GarageQual,5.447071
GarageCond,GarageCond,5.447071
GarageYrBlt,GarageYrBlt,5.447071


In [ ]:
# Exclusão de colunas com dados acima de 20% faltantes
cols = ['MiscFeature', 'PoolQC', 'Fence', 'Alley', 'FireplaceQu']
df = df.drop(cols, axis=1)

In [ ]:
# Tratando dados faltantes 
# Fillna substitui dados faltantes por algum valor de interesse. FFill pega o valor a frente e aplica ao registro faltante
# backfill oposto do ffill, backfill pega o valor anterior
df = df.fillna(method='ffill')

In [ ]:
# Checa se todos os dados faltantes foram preenchidos 
percent_missing = df.isnull().sum() * 100 / len(df) 
missing_value_df = pd.DataFrame({'column_name': df.columns, 
                                 'percent_missing': percent_missing})
missing_value_df.sort_values(['percent_missing'], ascending=False) 

,column_name,percent_missing
Id,Id,0.0
FullBath,FullBath,0.0
Fireplaces,Fireplaces,0.0
Functional,Functional,0.0
TotRmsAbvGrd,TotRmsAbvGrd,0.0
KitchenQual,KitchenQual,0.0
KitchenAbvGr,KitchenAbvGr,0.0
BedroomAbvGr,BedroomAbvGr,0.0
HalfBath,HalfBath,0.0
BsmtHalfBath,BsmtHalfBath,0.0


In [ ]:
# Feature Engineering 

df['IdadeCasa'] = 2022 - df['YearBuilt']                        # Cria variável idade da casa (ano atual - ano de construção)
df['TempoAteVenda'] = df['YearBuilt'] - df['YrSold']            # Cria variável de tempo de venda (ano da construção - ano de venda )
df['TempoAteReforma'] = df['YearBuilt'] - df['YearRemodAdd']    # Cria variável tempo até a reforma (ano de construção - ano de reforma)
df['TempoDaReforma'] = 2022 - df['YearRemodAdd']                # Cria variavel tempo da reforma (ano atual - ano da reforma) 
df['IdadeGarage'] = 2022 - df['GarageYrBlt']                    # Cria variavel idade da garagem (ano atual - ano de construção da garagem) 

In [ ]:
# Transforma variáveis Dummies 
df1 = pd.get_dummies(df, columns=
                     ['LotConfig',
                     'HouseStyle',
                     'Foundation',
                     'Electrical',
                     'MSZoning',
                     'SaleType',
                     'GarageType',
                     'LotShape',
                     'ExterCond',
                     'Foundation',
                     'BsmtExposure',
                     'Functional',
                     'SaleCondition',
                     'Street',
                     'LandContour',
                     'LandSlope',
                     'BldgType',
                     'Heating',
                     'PavedDrive'], drop_first=False)
df1.shape

(2919, 162)

In [ ]:
# Lista as variáveis do tipo objeto para transformação em binária com Label Encoder
objlist = df1.select_dtypes(include='object').columns
print(objlist)

Index(['Utilities', 'Neighborhood', 'Condition1', 'Condition2', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual',
       'BsmtQual', 'BsmtCond', 'BsmtFinType1', 'BsmtFinType2', 'HeatingQC',
       'CentralAir', 'KitchenQual', 'GarageFinish', 'GarageQual',
       'GarageCond'],
      dtype='object')


In [ ]:
# Transforma as variáveis objeto em binárias com LE
le = LabelEncoder()
for var in objlist:
  df1[var] = le.fit_transform(df1[var].astype(str))

In [ ]:
df1.shape

(2919, 162)

In [ ]:
# Separação em treino e teste

train = df1[df1['SalePrice'] != 0]
test = df1[df1['SalePrice'] == 0]

test.drop('SalePrice', axis=1, inplace=True)
train.shape, test.shape

((1460, 162), (1459, 161))

In [ ]:
X = train.drop('SalePrice', axis=1)
y = train['SalePrice']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [ ]:
# baixa os algoritmos 
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from lightgbm import LGBMRegressor

#### Random Forest

In [ ]:
rf_regressor = RandomForestRegressor(n_estimators=100,  # Numero de árvores, padrão 100
                                    criterion='squared_error', # Função que mede a qualidade da separação dos nós das arvores, padrão 'friedman_mse'
                                    max_depth=None, # Profundidae máxima da árvore, padrão None
                                    min_samples_split=2, # Numero minimo de separação de um nó interno, padrão 2
                                    min_samples_leaf=1, # Numero minimo de separação de um nó folha, padrão 1, ajuda a suavizar o modelo.
                                    min_weight_fraction_leaf=0.0, # A fração ponderada mínima da soma total de pesos (de todas as amostras de entrada) necessária para estar em um nó folha. As amostras têm peso igual quando sample_weight não é fornecido.
                                    max_features='auto', # O número de recursos a serem considerados ao procurar a melhor divisão:
                                    max_leaf_nodes=None, # Define limite de crescimento das arvores, default None
                                    min_impurity_decrease=0.0, # Um nó será dividido se esta divisão induzir uma diminuição da impureza maior ou igual a este valor.
                                    bootstrap=True, 
                                    oob_score=False, 
                                    n_jobs=None, # Número de threads paralelos a serem usados ​​para treinamento (pode ser alterado no momento da previsão).
                                    random_state=None, # Semente de número aleatório para reprodução
                                    verbose=0, # Habilita saida do modelo durante treinamento
                                    warm_start=False, # Inicia o treinamento com os pesos já existentes da arvores anterior, caso contrário comece do zero.
                                    ccp_alpha=0.0, 
                                    max_samples=None)  

In [ ]:
rf_regressor.fit(X_train, y_train)

RandomForestRegressor()

In [ ]:
y_pred = rf_regressor.predict(X_test)

In [ ]:
def metrics(y_test, y_pred):
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    return mae, mse, rmse, r2

In [ ]:
metrics(y_test, y_pred)

(17155.054212328767, 744227624.1584736, 27280.535628144724, 0.8795673151240536)

#### Gradient Boosting

In [ ]:
gd_regressor = GradientBoostingRegressor( loss='squared_error', # Função de custo, ou seja, o que vai ser minimizado
                                        learning_rate=0.1, # Taxa de aprendizado, padrão 0.1
                                        n_estimators=100, # Numero de árvores, padrão 100
                                        subsample=1.0, # Fração de amostras usadas para treinamento das árvores, padrão 1.0
                                        criterion='friedman_mse',  # Função que mede a qualidade da separação dos nós das arvores, padrão 'friedman_mse'
                                        min_samples_split=2, # Numero minimo de separação de um nó interno, padrão 2
                                        min_samples_leaf=1, # Numero minimo de separação de um nó folha, padrão 1, ajuda a suavizar o modelo.
                                        min_weight_fraction_leaf=0.0, # A fração ponderada mínima da soma total de pesos (de todas as amostras de entrada) necessária para estar em um nó folha. As amostras têm peso igual quando sample_weight não é fornecido.
                                        max_depth=3, # Profundidade máxima dos estimadores de regressão individuais. A profundidade máxima limita o número de nós na árvore. Ajuste este parâmetro para melhor desempenho; o melhor valor depende da interação das variáveis de entrada.
                                        min_impurity_decrease=0.0, # Um nó será dividido se esta divisão induzir uma diminuição da impureza maior ou igual a este valor.
                                        random_state=None, # Semente de número aleatório para reprodução
                                        max_features=None, # O número de recursos a serem considerados ao procurar a melhor divisão:
                                        verbose=0, # Habilita saida do modelo durante treinamento
                                        max_leaf_nodes=None, # Define limite de crescimento das arvores, default None
                                        warm_start=False, # Inicia o treinamento com os pesos já existentes da arvores anterior, caso contrário comece do zero.
                                        validation_fraction=0.1, # Proporção de dados de treinamento a ser destinado a validação
                                        n_iter_no_change=None, # Número de iterações sem melhora para parar o treinamento
                                        tol=0.0001, # Tolerância para parar o treinamento
                                        ccp_alpha=0.0) # Valor de regularização para o critério de parada

In [ ]:
gd_regressor.fit(X_train, y_train)

GradientBoostingRegressor()

In [ ]:
y_pred = gd_regressor.predict(X_test)

In [ ]:
metrics(y_test, y_pred)

(15408.065078254907, 586880478.5156655, 24225.616163797888, 0.9050296046067332)

#### AdaBoosting

In [ ]:
ada_regressor = AdaBoostRegressor(base_estimator=None,
                                 n_estimators=100, # Numero de árvores, padrão 100
                                 learning_rate=0.1, # Taxa de aprendizado, padrão 0.1
                                 loss='linear', # Função de custo, padrão linear
                                 random_state=None) # Semente de número aleatório para reprodução

In [ ]:
ada_regressor.fit(X_train, y_train)

AdaBoostRegressor(learning_rate=0.1, n_estimators=100)

In [ ]:
y_pred = ada_regressor.predict(X_test)

In [ ]:
metrics(y_test, y_pred)

(24704.076497701397, 1392397910.0319307, 37314.848385487654, 0.774678857277818)

#### LightGBM

In [ ]:
lgbm_regressor = LGBMRegressor(boosting_type='gbdt',  # Gradient Boosting Decision, dart, goss ou rf
                                    num_leaves=31, # Maximo de nós folha da árvore, padrão 31
                                    max_depth=- 1, # Profundidae máxima da árvore, padrão -1
                                    learning_rate=0.1, # Taxa de aprendizado, padrão 0.1
                                    n_estimators=100, # Numero de árvores, padrão 100
                                    subsample_for_bin=200000, # numero de amostras para cada nó folha, padrão 200000
                                    objective='regression', # Função objetivo, padrão None, neste caso usaremos regressor
                                    class_weight=None, # Usado para classificação, atribui peso as classes. Muito útil em casos de classificação e classes desbalanceadas
                                    min_split_gain=0.0, # Redução de perda mínima necessária para fazer uma partição adicional em um nó folha da árvore.
                                    min_child_weight=0.001, # Soma mínima do peso da instância (Hessian) necessária em um filho (folha).
                                    min_child_samples=20, # Número mínimo de dados necessários em um filho (folha).
                                    subsample=1.0, # proporção de subamostra da instância de treinamento.
                                    subsample_freq=0, # Frequência da subamostra, <=0 significa que não há habilitação.
                                    colsample_bytree=1.0, # Proporção de subamostra de colunas ao construir cada árvore.
                                    reg_alpha=0.0,  # Termo de regularização L1 em ​​pesos.
                                    reg_lambda=0.0, # Termo de regularização L2 em pesos.
                                    random_state=None, # Semente de número aleatório para reprodução
                                    n_jobs=- 1, # Número de threads paralelos a serem usados ​​para treinamento (pode ser alterado no momento da previsão).
                                    importance_type='split', # O tipo de importância do recurso a ser preenchido em feature_importances_. Se 'dividido', o resultado contém o número de vezes que o recurso é usado em um modelo. Se 'ganho', o resultado contém ganhos totais de divisões que usam o recurso.
                                    ) 

In [ ]:
lgbm_regressor.fit(X_train, y_train)

LGBMRegressor(objective='regression')

In [ ]:
y_pred = lgbm_regressor.predict(X_test)

In [ ]:
metrics(y_test, y_pred)

(16679.192688156945, 721953991.5053582, 26869.201542013827, 0.8831716873554505)

#### Tuning de hiperparametros com Random Search 

In [ ]:
n_estimators = [100, 150, 200, 250, 350, 400]
min_samples_split = [2, 6, 10]
min_samples_leaf = [2, 4, 6, 8]
max_features = ['auto', 'sqrt', 'log2', None]

parametros = {
    'n_estimators':n_estimators,
    'min_samples_split':min_samples_split,
    'min_samples_leaf':min_samples_leaf,
    'max_features':max_features
}

In [ ]:
gb_tuned = RandomizedSearchCV(estimator=GradientBoostingRegressor(random_state=37), 
                              param_distributions=parametros, 
                              cv=5, 
                              n_iter=10, 
                              n_jobs=-1,
                              verbose=1)
gb_tuned.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


RandomizedSearchCV(cv=5, estimator=GradientBoostingRegressor(random_state=37),
                   n_jobs=-1,
                   param_distributions={'max_features': ['auto', 'sqrt', 'log2',
                                                         None],
                                        'min_samples_leaf': [2, 4, 6, 8],
                                        'min_samples_split': [2, 6, 10],
                                        'n_estimators': [100, 150, 200, 250,
                                                         350, 400]},
                   verbose=1)